<a href="https://colab.research.google.com/github/silver5938/NLP_Twitter_Sentiment/blob/main/NLP_Twitter_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**" 딥러닝을 이용한 트위터 감성 분석 project "**

1. 학습 데이터 : AI허브에서 가져온 감성 train 데이터를 사용
2. 테스트 데이터 : lucy api를 이용해 데이터 10,000건 추출

프로젝트 세팅

In [ ]:
# DATASET
DATASET_COLUMNS = ["seq", "age", "sex", "keywords","health", "sentiment1", "sentiment2","text"]
DATASET_ENCODING = "CP949"
TRAIN_SIZE = 40879


# SENTIMENT
불안 = '0'
슬픔 = '1'
분노 = '2'
기쁨 = '3'
상처 = '4'
당황 = '5'


# EXPORT


기타 설정(import)


In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 62 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595726 sha256=819ae0ad166dd22d45383bce2ec96aa6707a62e02de6bf8de4984a72e09fd3c3
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 9.1 MB/s 
     |████████████████████████████████| 769 kB 8.3 MB/s 
     |████████████████████████████████| 3.0 MB 60.9 MB/s 
     |████████████████████████████████| 895 kB 37.8 MB/s 


In [2]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-kx87emnb
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-kx87emnb
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=5d394828fd0b7b6421fbee2dd03cf9157c2aabfd58a32778827a5593c10a77e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-9e71eb90/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
#pytorch
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary as summary_
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#GPU 사용
device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

# DataFrame
import pandas as pd

# Matplot
import matplotlib as mpl  # 기본 설정 만지는 용도
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm  # 폰트 관련 용도
import seaborn as sns
import missingno as msno
from plotly import graph_objs as go
plt.style.use('seaborn')
sns.set(font_scale=2.5) # matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅하고, 일일이 graph 의 font size 를 지정할 필요 없이 seaborn 의 font_scale 을 사용하면 편함

%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


※ 한글 깨짐 현상을 해결하는 방법 ※

In [ ]:
# Google Colab matplotlib 한글 폰트 오류 문제 해결

# 1. 나눔 폰트 다운
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 2. 런타임 다시 시작



Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,873 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 160837 files and d

In [ ]:
# 3. matplotlib의 폰트를 Nanum 폰트로 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

파일 업로드 - drive연동했다면 할 필요 없음

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import files
uploaded = files.upload()

파일 불러오기 :  
한글은 utf-8형식이므로 깨지기 때문에 encoding='CP949' 로 설정 해 주어야 한다.

In [ ]:
# colab과 google Drive 연동을 위한 인증 및 네트워크 디렉토리 마운트
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/dataset/s_training.CSV', encoding='CP949')
df_test = pd.read_csv('/content/drive/MyDrive/dataset/s_test.CSV', encoding='CP949')

head 메소드를 이용해 데이터 확인

In [ ]:
df_test.head(5)

,i_docid,i_sitename,i_url,i_title,i_sdate,sentiment1
0,TW1419668235989880833,Twitter,https://twitter.com/KaiS2com/status/1419668235...,＂걱정돼서 전화했어요＂ 우도주막 걱정에 깜짝 전화한 에이스 카이ㅜㅜ(서윗) | tv...,20210726,기쁨
1,TW1419661732599844866,Twitter,https://twitter.com/HuffPostKorea/status/14196...,"""돈에 지배당한 것"" 서장훈이 쓰리잡으로 1억 모은 25살 의뢰인에게 한 말은 일리...",20210726,기쁨
2,TW1419672897426231297,Twitter,https://twitter.com/threadreaderapp/status/141...,"@aqualemine Namaste, please find the unroll he...",20210726,기쁨
3,TW1419667990836957184,Twitter,https://twitter.com/Angry82pigeon/status/14196...,@dullchip0 너무 좋은..생각이예요 모두 궁금한걸~?!?(슴췹님 그림 하나라...,20210726,기쁨
4,TW1419672897426231297,Twitter,https://twitter.com/threadreaderapp/status/141...,"@aqualemine Namaste, please find the unroll he...",20210726,기쁨


In [ ]:
df_train.head(5)

,seq,age,sex,keywords,health,sentiment1,sentiment2,text
0,2,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,회사에서 중요한 큰 프로젝트를 나 혼자 하게 되니 솔직히 두려워.
1,3,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,회사에 직장 상사가 무섭게 생겨서 친해지는 게 두려워.
2,4,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,이번에 처음으로 힘들게 들어간 직장인데 첫 직장이라서 그런지 너무 긴장돼.
3,5,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,직장에서 사람들과의 관계에서 안 좋아질까 걱정돼.
4,7,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,회의 시간에 나의 직급이 낮아서 그런지 의견을 무시하는 경향이 있어서 의견을 제시하...


데이터 갯수 확인

In [ ]:
train_len = len(df_train)
test_len = len(df_test)

In [ ]:
train_len

40879

In [ ]:
test_len

303

info 메소드 이용해 기본적인 데이터 정보 확인

In [ ]:
df_train.info()
df_train.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40879 entries, 0 to 40878
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   seq         40879 non-null  int64 
 1   age         40879 non-null  object
 2   sex         40879 non-null  object
 3   keywords    40879 non-null  object
 4   health      40879 non-null  object
 5   sentiment1  40879 non-null  object
 6   sentiment2  40879 non-null  object
 7   text        40879 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.5+ MB


seq           0
age           0
sex           0
keywords      0
health        0
sentiment1    0
sentiment2    0
text          0
dtype: int64

In [ ]:
df_test.info()
df_test.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   i_docid     303 non-null    object
 1   i_sitename  303 non-null    object
 2   i_url       303 non-null    object
 3   i_title     303 non-null    object
 4   i_sdate     303 non-null    int64 
 5   sentiment1  303 non-null    object
dtypes: int64(1), object(5)
memory usage: 14.3+ KB


i_docid       0
i_sitename    0
i_url         0
i_title       0
i_sdate       0
sentiment1    0
dtype: int64


데이터 null값 확인(1)

In [ ]:
for col in df_train.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

column:        seq	 Percent of NaN value: 0.00%
column:        age	 Percent of NaN value: 0.00%
column:        sex	 Percent of NaN value: 0.00%
column:   keywords	 Percent of NaN value: 0.00%
column:     health	 Percent of NaN value: 0.00%
column: sentiment1	 Percent of NaN value: 0.00%
column: sentiment2	 Percent of NaN value: 0.00%
column:       text	 Percent of NaN value: 0.00%


In [ ]:
for col in df_test.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

column:    i_docid	 Percent of NaN value: 0.00%
column: i_sitename	 Percent of NaN value: 0.00%
column:      i_url	 Percent of NaN value: 0.00%
column:    i_title	 Percent of NaN value: 0.00%
column:    i_sdate	 Percent of NaN value: 0.00%
column: sentiment1	 Percent of NaN value: 0.00%


데이터 정제(Data Cleaning)_1

In [5]:
df_train.drop_duplicates(subset = ['text'], inplace=True) # text 열에서 중복인 내용이 있다면 중복 제거
df_train['text'] = df_train['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
df_train['text'] = df_train['text'].str.replace('^ +', "") # 공백은 empty 값으로 변경
df_train['text'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
df_train = df_train.dropna(how='any') # Null 값 제거
print('전처리 후 학습용 샘플의 개수 :',len(df_train))

2021-08-05 09:54:47,276 : INFO : NumExpr defaulting to 2 threads.


전처리 후 학습용 샘플의 개수 : 39415


In [6]:
df_test.drop_duplicates(subset = ['i_title'], inplace=True) # i_title 열에서 중복인 내용이 있다면 중복 제거
df_test['i_title'] = df_test['i_title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
df_test['i_title'] = df_test['i_title'].str.replace('^ +', "") # 공백은 empty 값으로 변경
df_test['i_title'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
df_test = df_test.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(df_test))

전처리 후 테스트용 샘플의 개수 : 297


In [ ]:
df_train.sample(n=10)

,seq,age,sex,keywords,health,sentiment1,sentiment2,text
1860,2364,청년,남성,대인관계,해당없음,기쁨,안도,오늘 차를 끌고 친구랑 여행을 가는데 졸다가 하마터면 사고 날 뻔했어
27947,35154,청년,여성,"연애, 결혼, 출산",해당없음,기쁨,안도,이번에 같이 인사 드리러 갔을 때 결혼 이야기가 나와서 부모님이 허락해주셔서 정말 기뻤어
24744,31117,노년,여성,"건강, 죽음",만성질환 무,불안,혼란스러운,옆집 영란 댁은 장례준비를 철저히 계획했던데 그에 비해 나는 아무것도 준비한 게 없...
30679,38578,중년,여성,"대인관계(부부, 자녀)",해당없음,기쁨,신이 난,취업한 딸이 첫 월급이라고 하면서 백만 원을 주었어 딸이 다 큰 것 같아서 뿌듯해
31190,39216,노년,남성,재정,만성질환 유,분노,짜증내는,지금 재정 상태가 좋지 않아서 작은 일로도 아내와 크게 다투게 되는 경우가 많아
27627,34764,노년,여성,재정,만성질환 유,상처,상처,내가 암 수술로 병원에 있는 동안 딸이 내 통장의 돈을 빼갔어 너무 충격이야
20375,25700,청년,남성,대인관계,해당없음,불안,취약한,친구들이 나보고 다 좋은데 왜 연락이 잘 안 되냐고 성화야
36865,46364,청소년,여성,가족관계,해당없음,당황,남의 시선을 의식하는,우리 가족은 매일 싸워서 창피해 밖에서도 싸운다니까
38084,47934,중년,여성,건강,해당없음,분노,노여워하는,요즘 몸이 안 좋아 보약을 사두었는데 그걸 남편이 다 먹어버렸어
5066,6318,중년,여성,"직장, 업무 스트레스",해당없음,슬픔,실망한,그 인간이 원하는 게 내 퇴사가 아닐까 싶어


데이터 확인

In [ ]:
filteredData = df_train['sentiment1'].isin(['기쁨','분노','불안','슬픔','상처','당황'])
filterdd = df_train[~filteredData]

In [ ]:
filterdd

,seq,age,sex,keywords,health,sentiment1,sentiment2,text


In [ ]:
temp = df_train.groupby('sentiment1').count()['seq'].reset_index().sort_values(by='seq',ascending=False)
temp.style.background_gradient(cmap='Purples')

,sentiment1,seq
3,불안,7107
2,분노,6671
5,슬픔,6659
0,기쁨,6455
4,상처,6421
1,당황,6102


In [ ]:
fig = go.Figure(go.Funnelarea(
    text =temp.sentiment1,
    values = temp.seq,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

감성(sentiment1) 라벨링

In [7]:
df_train.loc[(df_train['sentiment1'] == "불안"), 'sentiment1'] = 0  #불안 => 0
df_train.loc[(df_train['sentiment1'] == "슬픔"), 'sentiment1'] = 1  #슬픔 => 1
df_train.loc[(df_train['sentiment1'] == "분노"), 'sentiment1'] = 2  #분노 => 2
df_train.loc[(df_train['sentiment1'] == "기쁨"), 'sentiment1'] = 3  #기쁨 => 3
df_train.loc[(df_train['sentiment1'] == "상처"), 'sentiment1'] = 4  #상처 => 4
df_train.loc[(df_train['sentiment1'] == "당황"), 'sentiment1'] = 5  #당황 => 5

In [8]:
data_list = []
for q, label in zip(df_train['text'], df_train['sentiment1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['회사에서 중요한 큰 프로젝트를 나 혼자 하게 되니 솔직히 두려워', '0']
['내가 육십 세부터 받을 수 있는 연금 금액이 실망스러워 ', '1']
['내가 업무 때문에 직장동료에게 전화를 했는데 글쎄 직장동료 애인이 나를 내연녀로 오해했대', '5']
['결혼한 지 오 년째 아이가 생기지 않아서 눈물이 나', '1']
['새로운 부서에 발령을 받았는데 명예퇴직 대상자만 모아둔 곳이라 우울해', '1']
['모든 가족의 빚을 나 혼자 갚고 있어서 고립된 느낌이야 아무도 나를 도와주지 않아', '4']
['나 매우 화가 나', '2']


데이터 정제(Data Cleaning)_2

In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 448 kB 44.1 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-08-03 23:04:21--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
#불용어 제거
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']


In [ ]:
X_train = []
for sentence in df_train['text']:
    temp_X = mecab.morphs(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [ ]:
X_train[:10]

[['회사', '에서', '중요', '큰', '프로젝트', '나', '혼자', '하', '게', '되', '니', '솔직히', '두려워'],
 ['회사', '직장', '상사', '무섭', '게', '생겨서', '친해', '지', '게', '두려워'],
 ['이번',
  '처음',
  '힘들',
  '게',
  '들어간',
  '직장',
  '인데',
  '첫',
  '직장',
  '라서',
  '그런지',
  '너무',
  '긴장',
  '돼'],
 ['직장', '에서', '사람', '관계', '에서', '안', '좋', '아', '질까', '걱정', '돼'],
 ['회의',
  '시간',
  '나',
  '직급',
  '낮',
  '아서',
  '그런지',
  '의견',
  '을',
  '무시',
  '하',
  '경향',
  '있',
  '어서',
  '의견',
  '을',
  '제시',
  '하',
  '것',
  '힘들',
  '어'],
 ['회사',
  '에서',
  '부장',
  '님',
  '께',
  '보고',
  '올렸',
  '는데',
  '내',
  '실수',
  '부분',
  '있',
  '어서',
  '걸릴까',
  '봐',
  '걱정',
  '야'],
 ['내일',
  '회사',
  '면접',
  '을',
  '보',
  '러',
  '는데',
  '내',
  '틱',
  '증상',
  '있',
  '어서',
  '떨어질까',
  '봐',
  '걱정',
  '야'],
 ['내',
  '새로운',
  '직장',
  '에서',
  '적응',
  '을',
  '할',
  '수',
  '있',
  '을지',
  '모르',
  '겠',
  '어',
  '불안',
  '해'],
 ['평소',
  '날',
  '좋',
  '지',
  '않',
  '게',
  '보',
  '시',
  '팀장',
  '님',
  '있',
  '는데',
  '내',
  '아이디어',
  '말',
  '하',
  '면',
  '하찮',
  '게',
  '여기',
  '실

sklearn을 이용한 Train & Test 데이터 나누기

In [9]:
#train & test 데이터로 나누기
                                                     
dataset_train, dataset_test = train_test_split(data_list, test_size=0.20, random_state=0)

In [10]:
print(len(dataset_train))
print(len(dataset_test))

31532
7883


KoBERT 입력 데이터로 만들기

데이터를 train data와 test data로 나누었다면 각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등을 해주어야 한다. 예시 코드에 입력데이터의 형태가 되도록 해주는 클래스가 있어서 동일하게 코드를 작성해주었다.

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

하이퍼 파라미터들을 조정해준다. Batch size는 16, epochs는 7, learning rate는 5e-5로 설정

In [12]:
# Setting parameters
max_len = 128
batch_size = 16
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

토크나이징(tokenizing)

In [13]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
data_train[0]

(array([   2, 4626, 7828, 4628, 7086, 2047, 5156, 6896, 4598, 7342, 7880,
        1375, 3129, 5859, 3166, 6273, 6542, 7096, 6844,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(20, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

torch형식의 dataset 만들기

In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

KoBERT 학습모델 만들기

In [49]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,#DROPOUT
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

KoBERT 모델 학습시키기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
  

모델 저장

In [ ]:
PATH1 = '/content/drive/MyDrive/model/model_sentiment1.pt'
torch.save(model, PATH1)

In [ ]:
PATH2 = '/content/drive/MyDrive/model/model_sentiment2.pt'
torch.save(model.state_dict(), PATH2)

In [ ]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
from torchvision import models
model2 = models.vgg16()
print(model2)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

모델 불러오기

In [19]:
model = torch.load('/content/drive/MyDrive/model/model_sentiment1.pt')
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

LUCY API 데이터 정확도 검증

In [37]:
df_test.loc[(df_test['sentiment1'] == "불안"), 'sentiment1'] = 0  #불안 => 0
df_test.loc[(df_test['sentiment1'] == "슬픔"), 'sentiment1'] = 1  #슬픔 => 1
df_test.loc[(df_test['sentiment1'] == "분노"), 'sentiment1'] = 2  #분노 => 2
df_test.loc[(df_test['sentiment1'] == "기쁨"), 'sentiment1'] = 3  #기쁨 => 3
df_test.loc[(df_test['sentiment1'] == "상처"), 'sentiment1'] = 4  #상처 => 4
df_test.loc[(df_test['sentiment1'] == "당황"), 'sentiment1'] = 5  #당황 => 5
df_test.loc[(df_test['sentiment1'] == "중립"), 'sentiment1'] = 6  #중립 => 6

In [58]:
temp = df_test.groupby('sentiment1').count()['i_docid'].reset_index().sort_values(by='i_docid',ascending=False)
temp.style.background_gradient(cmap='Purples')

,sentiment1,i_docid
3,3,126
6,6,90
1,1,23
2,2,21
5,5,15
0,0,14
4,4,8


In [38]:
data_list2 = []
for q, label in zip(df_test['i_title'], df_test['sentiment1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list2.append(data)

In [70]:
df_test2_num = df_test[df_test['sentiment1'] == 6].index
df_test2 = df_test.drop(df_test2_num)
print(df_test2)

                   i_docid i_sitename  ...   i_sdate sentiment1
0    TW1419668235989880833    Twitter  ...  20210726          3
1    TW1419661732599844866    Twitter  ...  20210726          3
2    TW1419672897426231297    Twitter  ...  20210726          3
3    TW1419667990836957184    Twitter  ...  20210726          3
5    TW1419655983027027979    Twitter  ...  20210726          3
..                     ...        ...  ...       ...        ...
297  TW1419631097025949700    Twitter  ...  20210726          3
298  TW1419631078533328897    Twitter  ...  20210726          3
299  TW1419631036787347456    Twitter  ...  20210726          1
301  TW1419631022237315074    Twitter  ...  20210726          3
302  TW1419631014687612928    Twitter  ...  20210726          5

[207 rows x 6 columns]


In [71]:
data_list3 = []
for q, label in zip(df_test2['i_title'], df_test2['sentiment1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list3.append(data)

In [39]:
print(data_list2[0])
print(data_list2[200])


['걱정돼서 전화했어요 우도주막 걱정에 깜짝 전화한 에이스 카이ㅜㅜ서윗    방송 출처  우도주막  네이버 좋아요 예쁜 ', '3']
['와 인연 있었던 가수였는데오죽했으면  김동욱 올림픽논란 도쿄올림픽', '1']


In [40]:
data_test_lucy = BERTDataset(data_list2, 0, 1, tok, max_len, True, False)

In [72]:
data_test_lucy2 = BERTDataset(data_list3, 0, 1, tok, max_len, True, False)

In [41]:
data_test_lucy[0]

(array([   2,  880, 5876, 6553, 4064, 7874, 3498, 5859, 7276, 6149,  880,
        6896, 1341, 4064, 7828, 3290, 4635, 7096,    0, 6553, 7061, 2272,
        4586, 3498, 5859, 7276, 6149, 1470, 4207, 6999, 3396, 6489,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(33, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

In [42]:
test_lucy_dataloader = torch.utils.data.DataLoader(data_test_lucy, batch_size=batch_size, num_workers=5)

In [73]:
test_lucy_dataloader2 = torch.utils.data.DataLoader(data_test_lucy2, batch_size=batch_size, num_workers=5)

In [44]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,#DROPOUT
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [56]:
test_acc = 0.0

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_lucy_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
print("lucy test acc {}".format(test_acc / (batch_id+1)))


lucy test acc 0.28728070175438597


In [74]:
test_acc = 0.0

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_lucy_dataloader2)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
print("lucy test acc {}".format(test_acc / (batch_id+1)))


lucy test acc 0.41025641025641024


예측 모델(문장 테스트)


입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성

In [20]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

#input, output 산출 가능한 class
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("불안")
            elif np.argmax(logits) == 1:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 2:
                test_eval.append("분노")
            elif np.argmax(logits) == 3:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 4:
                test_eval.append("상처")
            elif np.argmax(logits) == 5:
                test_eval.append("당황")
            else :
                test_eval.append("중립")

        print(">> 입력한 내용의 감성은 " + test_eval[0] + " 입니다.")

using cached model


In [23]:
#질문형 감성 판단! F 입력시 종료
end = 1
while end == 1 :
    sentence = input("문장을 입력하세요 : ")
    if sentence == 'F' :
        break
    predict(sentence)
    print("\n")

문장을 입력하세요 : 너가 싫어
>> 입력한 내용의 감성은 분노 입니다.


문장을 입력하세요 : 아이가 아파
>> 입력한 내용의 감성은 슬픔 입니다.


문장을 입력하세요 : 공부를 더 열심히 했어야 했는데,,
>> 입력한 내용의 감성은 불안 입니다.


문장을 입력하세요 : 그사람이 날 찼어
>> 입력한 내용의 감성은 상처 입니다.


문장을 입력하세요 : 내일 여행이 기대된당
>> 입력한 내용의 감성은 기쁨 입니다.


문장을 입력하세요 : 그날의 일이 너무 부끄러워
>> 입력한 내용의 감성은 당황 입니다.


문장을 입력하세요 : 쏘스윗ㅜ
>> 입력한 내용의 감성은 슬픔 입니다.


문장을 입력하세요 : 너무 좋아 어떻게 해ㅜㅜㅜㅜㅜ
>> 입력한 내용의 감성은 당황 입니다.


문장을 입력하세요 : ㅈㄴ 너무 좋아
>> 입력한 내용의 감성은 기쁨 입니다.


문장을 입력하세요 : 아 ㅅㅂ 너무 좋아
>> 입력한 내용의 감성은 기쁨 입니다.


문장을 입력하세요 : 참 대단하다 대단해 다 해먹어라
>> 입력한 내용의 감성은 기쁨 입니다.


문장을 입력하세요 : 오늘따라 눈길이가
>> 입력한 내용의 감성은 불안 입니다.


문장을 입력하세요 : 뽐뿌온다
>> 입력한 내용의 감성은 분노 입니다.


문장을 입력하세요 : 니가 뭔데
>> 입력한 내용의 감성은 당황 입니다.


문장을 입력하세요 : 어떻게 되든 상관마
>> 입력한 내용의 감성은 슬픔 입니다.


문장을 입력하세요 : 카이 내 왕자님
>> 입력한 내용의 감성은 기쁨 입니다.


문장을 입력하세요 : 공주대접 받으려고 왔냐
>> 입력한 내용의 감성은 상처 입니다.


문장을 입력하세요 : 아주 왕자님 납셨네
>> 입력한 내용의 감성은 기쁨 입니다.


문장을 입력하세요 : F


In [77]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.
